In [ ]:
import os

In [ ]:
from data.dataset import Dataset
from data.feature.descriptor import DescriptorFeaturizer
from data.dataloader import DataLoader
from methods.ridge_method import RidgeMethod
from methods.bayesian_method import BayesianRidgeMethod
from methods.decision_tree_method import DecisionTreeMethod
from methods.elasticnet_method import ElasticNetMethod
from methods.mlp_method import MlpMethod
from methods.lasso_method import LassoMethod
from methods.random_forest_method import RandomForestMethod
from methods.knn_method import KnnMethod
from methods.lasso_lars_method import LassoLarsMethod
from methods.pca_method import PCAAnalysis

import numpy as np
from utils.visualize import visualize_energy

In [ ]:
import pandas as pd

In [ ]:
from methods.preprocessing.pca import PCAMethod
from methods.preprocessing.shift import ShiftingMethod
from methods.preprocessing.identity import IdentityMethod

In [ ]:
def rmse(y_true, y_pred, y_train):
    return np.sqrt(np.mean((y_true - y_pred) ** 2)) / np.std(y_train)

In [ ]:
methods = [RidgeMethod, BayesianRidgeMethod, DecisionTreeMethod, ElasticNetMethod, LassoMethod, RandomForestMethod, KnnMethod, LassoLarsMethod]

In [ ]:
featurizer = DescriptorFeaturizer()
#trimer_dataset = Dataset.from_file('xe3_50.xyz', 3, featurizer)
#energy_base = trimer_dataset[-1][1][0]

dimer_dataset = Dataset.from_file('dataset/xe2_50.xyz', 2, featurizer)
trimer_dataset = Dataset.from_file('dataset/xe3_50.xyz', 3, featurizer)
rand_trimer_dataset = Dataset.from_file('dataset/xe3_dataset_dft.xyz', 3, featurizer)

In [ ]:
n_dimer_train = 40
n_trimer_train = 40
n_rand_trimer_train = 5000

dimer_train, dimer_val = dimer_dataset.split(
    [list(range(n_dimer_train)),
     list(range(n_dimer_train, len(dimer_dataset)))])
trimer_train, trimer_val = trimer_dataset.split(
    [list(range(n_trimer_train)),
     list(range(n_trimer_train, len(dimer_dataset)))])
rand_trimer_train, rand_trimer_val = rand_trimer_dataset.split(
    [list(range(n_rand_trimer_train)), 
     list(range(n_rand_trimer_train, len(rand_trimer_dataset)))])
train = DataLoader([dimer_train, trimer_train, rand_trimer_train])
val = DataLoader([dimer_val, trimer_val, rand_trimer_val])
dimer_val = DataLoader([dimer_val])
trimer_val = DataLoader([trimer_val])
rand_trimer_val = DataLoader([rand_trimer_val])

In [ ]:
pca = IdentityMethod()
shift = IdentityMethod()#ShiftingMethod(-205862)

In [ ]:
train = DataLoader([dimer_train, trimer_train, rand_trimer_train])
val = DataLoader([dimer_val, trimer_val, rand_trimer_val])
train.X = pca.fit_preprocess(train.X)
train.y = shift.fit_preprocess(train.y)

# Methods selection

In [ ]:
data = pd.DataFrame(columns=['train%RMSE$_{dimer}$', 'train%RMSE$_{trimer}$', 'train%RMSE$_{rand_trimer}$', '%RMSE$_{dimer}$', '%RMSE$_{trimer}$', '%RMSE$_{rand_trimer}$'])

for method in methods:
    m = method(train)
    m.train()
    dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_dataset.X))) * 2
    trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_dataset.X))) * 3
    fig, ax = visualize_energy(dimer_dataset, dimer_energy - dimer_energy[-1], trimer_dataset, trimer_energy - trimer_energy[-1])
    train_dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_train.X))) * 2
    train_trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_train.X))) * 3
    train_rand_trimer_energy = shift.backward(m.predict(pca.preprocess(rand_trimer_train.X))) * 3
    val_dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_val.X))) * 2
    val_trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_val.X))) * 3
    val_rand_trimer_energy = shift.backward(m.predict(pca.preprocess(rand_trimer_val.X))) * 3
    data.loc[method.__name__] = [
        rmse(train_dimer_energy, dimer_train.y * 2, train.y[:n_dimer_train]), 
        rmse(train_trimer_energy, trimer_train.y * 3, train.y[50:50+n_trimer_train]), 
        rmse(train_rand_trimer_energy, rand_trimer_train.y * 3, train.y[100:100+n_rand_trimer_train]),
        rmse(val_dimer_energy, dimer_val.y * 2, train.y[:n_dimer_train]), 
        rmse(val_trimer_energy, trimer_val.y * 3, train.y[50:50+n_trimer_train]), 
        rmse(val_rand_trimer_energy, rand_trimer_val.y * 3, train.y[100:100+n_rand_trimer_train])]
    
    ax[0].set_title(method.__name__)

# MLP test

In [ ]:
methods = [MlpMethod(dataloader=train, hidden=[128], dropout=0.1, wd=1e-2, batch_size=64, epochs=100, lr=0.005),
           MlpMethod(dataloader=train, hidden=[128, 64], dropout=0.1, wd=1e-2, batch_size=64, epochs=100, lr=0.005)]

In [ ]:
data = pd.DataFrame(columns=['train%RMSE$_{dimer}$', 'train%RMSE$_{trimer}$', 'train%RMSE$_{rand_trimer}$', '%RMSE$_{dimer}$', '%RMSE$_{trimer}$', '%RMSE$_{rand_trimer}$'])

for n, method in zip(['MlpMethod1', 'MlpMethod2'],methods):
    m = method#(train)
    m.train()
    dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_dataset.X))) * 2
    trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_dataset.X))) * 3
    fig, ax = visualize_energy(dimer_dataset, dimer_energy - dimer_energy[-1], trimer_dataset, trimer_energy - trimer_energy[-1])
    train_dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_train.X))) * 2
    train_trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_train.X))) * 3
    train_rand_trimer_energy = shift.backward(m.predict(pca.preprocess(rand_trimer_train.X))) * 3
    val_dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_val.X))) * 2
    val_trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_val.X))) * 3
    val_rand_trimer_energy = shift.backward(m.predict(pca.preprocess(rand_trimer_val.X))) * 3
    data.loc[n] = [
        rmse(train_dimer_energy, dimer_train.y * 2, train.y[:n_dimer_train]), 
        rmse(train_trimer_energy, trimer_train.y * 3, train.y[50:50+n_trimer_train]), 
        rmse(train_rand_trimer_energy, rand_trimer_train.y * 3, train.y[100:100+n_rand_trimer_train]),
        rmse(val_dimer_energy, dimer_val.y * 2, train.y[:n_dimer_train]), 
        rmse(val_trimer_energy, trimer_val.y * 3, train.y[50:50+n_trimer_train]), 
        rmse(val_rand_trimer_energy, rand_trimer_val.y * 3, train.y[100:100+n_rand_trimer_train])]
    
    ax[0].set_title(n)

# Ridge test

In [ ]:
alphas = [np.geomspace(alpha, alpha, 1) for alpha in np.geomspace(1e-16, 1e2, 37)]

In [ ]:
data = pd.DataFrame(columns=['%RMSE$_{dimer, train}$', '%RMSE$_{trimer, train}$', '%RMSE$_{rand\_trimer, train}$', '%RMSE$_{dimer, val}$', '%RMSE$_{trimer, val}$', '%RMSE$_{rand\_trimer, val}$'])

for alpha in alphas:
    m = RidgeMethod(train, alpha)
    m.train()
    dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_dataset.X))) * 2
    trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_dataset.X))) * 3
    fig, ax = visualize_energy(dimer_dataset, dimer_energy - dimer_energy[-1], trimer_dataset, trimer_energy - trimer_energy[-1])
    train_dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_train.X))) * 2
    train_trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_train.X))) * 3
    train_rand_trimer_energy = shift.backward(m.predict(pca.preprocess(rand_trimer_train.X))) * 3
    val_dimer_energy = shift.backward(m.predict(pca.preprocess(dimer_val.X))) * 2
    val_trimer_energy = shift.backward(m.predict(pca.preprocess(trimer_val.X))) * 3
    val_rand_trimer_energy = shift.backward(m.predict(pca.preprocess(rand_trimer_val.X))) * 3
    data.loc[alpha[0]] = [
        rmse(train_dimer_energy, dimer_train.y * 2, train.y[:n_dimer_train]), 
        rmse(train_trimer_energy, trimer_train.y * 3, train.y[50:50+n_trimer_train]), 
        rmse(train_rand_trimer_energy, rand_trimer_train.y * 3, train.y[100:100+n_rand_trimer_train]),
        rmse(val_dimer_energy, dimer_val.y * 2, train.y[:n_dimer_train]), 
        rmse(val_trimer_energy, trimer_val.y * 3, train.y[50:50+n_trimer_train]), 
        rmse(val_rand_trimer_energy, rand_trimer_val.y * 3, train.y[100:100+n_rand_trimer_train])]
    
    ax[0].set_title(f'alpha = {alpha[0]}')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(dpi=200)
ax.set(xscale="log", yscale="log")
ax.set_title('Regularization Parameter Selection')
ax.set_xlabel('$\lambda$')
ax.set_ylabel('%RMSE')
ax.set_xticks(np.geomspace(1e-18, 1e2, 11))
#ax.set_xticklabels(np.geomspace(1e-18, 1e2, 21))
sns.lineplot(data=data)

# RMSE result

In [ ]:
data